In [1]:
from dotenv import load_dotenv
load_dotenv()
import vectrix
import sys
sys.path.append('..')

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Web Page Indexing and Vectorization 👀

This Jupyter notebook contains a script that performs indexing and vectorization of web page contents. The primary purpose of this script is to crawl through a specified web page, extract the textual contents, and subsequently store these contents as vector objects in a database.

The vectorized information can then be utilized in a Retrieval-Augmented Generation (RAG) flow to answer questions using a Language Model (LLM). This process enables the creation of a more context-aware and responsive system, capable of providing detailed responses based on the indexed and vectorized information from the web page.

The notebook is structured in a step-by-step manner, guiding you through the process of web page crawling, text extraction, vectorization, and storage in a database. Each step is accompanied by detailed explanations and code snippets to provide a comprehensive understanding of the process.

## Web Crawler and Content Extractor

This code implements a web crawler and content extractor that:

1. Extracts URLs from the given HTML content, filtering for the same domain and validating the URLs. ✅
2. Crawls a website starting from a given URL, iteratively processing and extracting links from each page. ✅
3. Returns a mist of HTML documents extracted from the website ✅

The code displays the source URL of each processed page and the total number of pages in the extracted content.

In [2]:
crawler = vectrix.Crawler("https://vectrix.ai", max_pages=20)
site_pages = crawler.extract()

Fetching pages: 100%|##########| 11/11 [00:02<00:00,  5.26it/s]


In [3]:
len(site_pages)

18

In [4]:
print(site_pages[8].content['text'])

Advice
Vectrix Advice Services: Collaborative AI Solutions for Your Business
AI Consultation and Strategic Advice
Vectrix provides expert consultations to help businesses align their AI initiatives with their broader business strategies. This includes offering guidance on how to leverage AI for maximum impact, identifying opportunities for AI implementation, and advising on best practices for integrating AI into existing business processes.
AI Readiness Assessment
This service involves a comprehensive evaluation of a business's current systems and infrastructure to determine their readiness for AI integration. Vectrix assesses the potential for AI application within the organization and provides recommendations on how to prepare for and implement AI solutions effectively.
Data Readiness and Optimization
Recognizing the crucial role of data in AI deployments, Vectrix advises businesses on how to prepare their data for AI use. This includes assessing the current data infrastructure, advi

## Data Preprocessing and Chunking
In this step we will split all the extracted web pages into logical chunks. 

➡️ We will use the [trafilatura](https://trafilatura.readthedocs.io/en/latest/) library to extract the main content of the web pages. It will return the main content of the page, the title, and the meta description.

➡️ We will pipe this to another splitter to further cut the sections into smaller chunks if they are too large. For this we use Langchains 

➡️  Also we will attach an LLM to the chain to ignore chunks that are not relevant, for example: navigation bars, footers, etc.



### Chunking and metadata extraction
Using the functions below we extract the medata and devide the text into chunks. 

In [5]:
chunker = vectrix.Webchunker(site_pages)
chunks = chunker.chunk_content(chunk_size=500, chunk_overlap=50)

In [6]:
len(chunks)

23

In [7]:
print(chunks[18].dict())

{'id': None, 'metadata': {'source': 'https://vectrix.ai/blog-post/advanced-applications-and-future-trends-in-entity-analysis', 'title': 'Vectrix', 'language': 'en'}, 'page_content': "can unlock new insights and make smarter decisions, enhancing our lives and business practices. What You Will Learn When Reading the Full Blog Post In the full blog post, you'll explore how entity analysis revolutionizes healthcare, finance, and legal services by extracting meaningful insights from data. You'll learn about real-world applications and the technical implementations driving these innovations. Additionally, discover future trends like real-time processing, multimodal analysis, and the integration of generative AI that promise to enhance personalization and decision-making in various fields. Whether you're looking for a technical deep dive or practical examples, the full post offers a comprehensive understanding of the potential and future of entity analysis.", 'type': 'Document'}


In [8]:
print(chunks[18].json(indent=2))

{
  "id": null,
  "metadata": {
    "source": "https://vectrix.ai/blog-post/advanced-applications-and-future-trends-in-entity-analysis",
    "title": "Vectrix",
    "language": "en"
  },
  "page_content": "can unlock new insights and make smarter decisions, enhancing our lives and business practices. What You Will Learn When Reading the Full Blog Post In the full blog post, you'll explore how entity analysis revolutionizes healthcare, finance, and legal services by extracting meaningful insights from data. You'll learn about real-world applications and the technical implementations driving these innovations. Additionally, discover future trends like real-time processing, multimodal analysis, and the integration of generative AI that promise to enhance personalization and decision-making in various fields. Whether you're looking for a technical deep dive or practical examples, the full post offers a comprehensive understanding of the potential and future of entity analysis.",
  "type": 

### NER Extraction Pipeline
Here we will use langchain and and LLM to extract the Named Entities from the text.

In [9]:
extractor = vectrix.Extract('Replicate', 'meta/meta-llama-3-70b-instruct')
results = extractor.extract(chunks)

Extracting entities: 100%|██████████| 23/23 [00:26<00:00,  1.14s/it]


In [10]:
print(results[4].dict()['metadata'])

{'source': 'https://vectrix.ai/platform', 'title': 'Vectrix - Platform', 'description': 'Our platform is your foundation for building unique generative AI applications that can enhance customer service, increase sales, and automate tasks. Dive into the extensive capabilities of our adaptable platform and discover how it can cater to your specific use case, integrating the transformative potential of AI directly into your business operations.', 'language': 'en', 'uuid': 'f8b4268c-2e8f-4ab9-a6aa-e16c959e4a9a', 'NER': {'entity_list': [{'entity_type': 'concept', 'entity_name': 'Small Language Models'}, {'entity_type': 'concept', 'entity_name': 'Large Language Models'}, {'entity_type': 'concept', 'entity_name': 'Artificial Intelligence'}, {'entity_type': 'location', 'entity_name': 'Virtual Private Cloud'}, {'entity_type': 'organization', 'entity_name': 'Apple Inc.'}], 'language': 'English', 'category': 'Artificial Intelligence'}}


In [11]:
# Show the memory usage of this notebook
import os
import psutil
process = psutil.Process(os.getpid())
print("Memory used: ", process.memory_info().rss / 1024 ** 2, "MB")

Memory used:  261.703125 MB


## Storing the result in a Weaviate (cluster)

### Initialize the Vector store and check that all the required modules are installed

Download the Docker compose file if needed
```bash
curl -o docker-compose.yml "https://configuration.weaviate.io/v2/docker-compose/docker-compose.yml?cohere_key_approval=yes&generative_anyscale=false&generative_aws=false&generative_cohere=false&generative_mistral=false&generative_octoai=false&generative_ollama=false&generative_openai=false&generative_palm=false&media_type=text&modules=modules&ner_module=false&qna_module=false&ref2vec_centroid=false&reranker_cohere=true&reranker_cohere_key_approval=yes&reranker_transformers=false&runtime=docker-compose&spellcheck_module=true&spellcheck_module_model=pyspellchecker-en&sum_module=false&text_module=text2vec-cohere&weaviate_version=v1.25.4&weaviate_volume=named-volume"
```

Make sure to set the persistent directory to the correct value:
```bash
    volumes:
    - ~/weaviate_data:/var/lib/weaviate
```

Also configure the Cohere API key:
```bash
environment:
      SPELLCHECK_INFERENCE_API: 'http://text-spellcheck:8080'
      COHERE_APIKEY: ***
```

In [ ]:
from vectrix.db.weaviate import Weaviate, VectorDocument

weaviate = Weaviate()

In [ ]:
weaviate.create_collection(name='Vectrix', 
                           embedding_model='Ollama', 
                           model_name="mxbai-embed-large:335m",
                           model_url="http://host.docker.internal:11434")

In [ ]:
print(weaviate.list_collections())

In [ ]:
weaviate.set_colleciton(name='Vectrix')

In [ ]:
data_to_vectorize = []

for result in results:
    data_to_vectorize.append(
        VectorDocument(
            title=result.metadata["title"],
            url=result.metadata["source"],
            content=result.page_content,
            type="webpage",
            NER=str(result.metadata["NER"]),
        )
    )

weaviate.add_data(data_to_vectorize)

In [ ]:
retriever = weaviate.get_retriever()
retriever.invoke('Who are the Vectrix founders ?')

In [ ]:
weaviate.remove_collection("Vectrix")

In [ ]:
weaviate.info()

In [ ]:
weaviate.close()